# FP
Start writing all my logic in jupyter notebook in order to run them from anaconda! <br>
** IMPORTANT ** <br>
the network cell should only be intialize once! otherwise the program start creating indexes for the variables!!!! 

## index: 
<a id='index'/>

1. READ DATA 
    * Class 
    * files 
2. [Model](#model)
3. [Train](#tr) 
4. [Evaluate](#ev) 
5. [Test](#ts) 
6. [Other](#o)

[go back](#index)


to do:
* make work the classes   - DECLARED! I NEED TO TEST IT; NEW FILE! 
* implement the reporting logic 
* idea: counting the number of elements - new column in pandas
* one drive not working


In [1]:
import pandas as pd 
import tensorflow as tf 
import numpy as np 

import requests
import json
import sys
import os
import time
from types import *
from collections import Counter
from datetime import datetime


In [2]:
LOG        = "../../_zfp/LOG.txt"
LOGDIR     = "../../_zfp/data/my_graph/"
LOGDAT     = "../../_zfp/data/"

DESC       = "FRFLO"

DSJ_T      = "/data_jsont.txt"
DSJ        = "/data_json.txt"
DSC        = "/TFFRFLO_ALSN.csv"   
DC         = "/colcom.csv"
DL         = "/datal.csv"

LAB_DS     = LOGDAT + DESC + DL  #"../../_zfp/data/FRFLO/datal.csv"
COL_DS     = LOGDAT + DESC + DC  #"../../_zfp/data/FRFLO/colcom.csv"
ALL_DSJ    = LOGDAT + DESC + DSJ #"../../_zfp/data/FRFLO/dataXX.csv"

ALL_DSJ    = LOGDAT + DESC + DSJ_T

ALL_DS     = LOGDAT + DESC + DSC #"../../_zfp/data/FRFLO/TFFRFLO_ALSN.csv"

# Read Data

In [3]:
#read data => class - COM or FP input // output - CAT 100 or 4! 
class fpDataModel:
    def __init__(self, path, norm, batch_size, dType, labelCol, dataCol = 4, nC=100, nRange=1 , toList = True):
        self.path           = path
        self.norm           = norm
        self.batch_size     = batch_size
        self.dType          = dType
        self.labelCol       = labelCol
        self.dataCol        = dataCol
        self.nC             = nC
        self.nRange         = nRange
        self.toList         = toList 

    def next_batch(self, num, data, labels):
        idx = np.arange(0 , len(data))
        np.random.shuffle(idx)
        idx = idx[:num]
        data_shuffle = [data[i] for i in idx]
        labels_shuffle = [labels[i] for i in idx]

        return np.asarray(data_shuffle), np.asarray(labels_shuffle)
    
    def classif(self, df):
        if( df < 40 ): return [0,0,0,1] 
        elif( df >= 40 and df < 60 ): return [0,0,1,0]
        elif( df >= 60 and df < 90 ): return [0,1,0,0] 
        elif( df >= 90 ): return [1,0,0,0] 
    def declassif(self, df): 
        if  ( df == [0,0,0,1] ):   return 1 
        elif( df == [0,0,1,0] ):   return 2
        elif( df == [0,1,0,0] ):   return 3  
        elif( df == [1,0,0,0] ):   return 4      
    def regress(self, df): #
        return [df]
    
    def classifN(self, df):
        listofzeros = [0] * self.nC
        dfIndex = df//self.nRange
        # print('{} and {}', (df,dfIndex))
        if dfIndex < self.nC:
            listofzeros[dfIndex] = 1 
        return listofzeros
    
    def deClassifN(self, df, val = 1 ):
        return df.index(val)
    
    def split_lab_dat(self, dst):
        cat  = dst.loc[:, self.labelCol]
        dat  = dst.iloc[:, self.dataCol:]
        if (self.toList): 
            cat = cat.as_matrix().tolist()
            dat = dat.as_matrix().tolist()
        return {'label' : cat, 'data' : dat}
    
    #Get Data
    def get_data(self, typeSep = True, pathA = "", filter = ""):
        if pathA != "":
            dst =  pd.read_csv( tf.gfile.Open(pathA), sep=None, skipinitialspace=True,  engine="python")
        else: 
            dst =  pd.read_csv( tf.gfile.Open(self.path), sep=None, skipinitialspace=True,  engine="python")
        
        dst = dst.fillna(0)
        
        if filter == '>23':
            dst = dst[dst["FP"]>23]
        elif filter == '>60':
            dst = dst[dst["FP"]>60]


        if self.norm != "":
            cat_n  = dst.loc[:,'FP'] 
            dst['FP'] = self.normalization( cat_n )

        if   self.dType == 'class':       # Classification in 4 categories
            dst.insert(2, 'FP_C', dst['FP'].map(lambda x: self.classif(x)))
        elif self.dType == 'reg':       # Regression
            dst.insert(2, 'FP_R', dst['FP'].map(lambda x: self.regress(x)))
        elif self.dType == 'classN':    # Classification in N categories  
            dst.insert(2, 'FP_C', dst['FP'].map(lambda x: self.classifN(x))) 
        
        self.dst = dst

        # 3 if no type and 4 if type
        if typeSep == True:
            dst_tmp = [rows for _, rows in dst.groupby('Type')]
            data_e  = self.split_lab_dat(dst_tmp[0])
            data_t  = self.split_lab_dat(dst_tmp[1])
            return data_t, data_e
        else :   return  self.split_lab_dat(dst_tmp[0])
    
    
    
    def get_data2(self, colu="", datu=""):
        pass
    
    def set_columns(self, url ):        # set the main data frame from the class: 
        columns_path = url
        self.col_df = pd.read_csv(columns_path, index_col=0, sep=',', usecols=[0,1,2,3])
        return(len(self.col_df))
        
    def feed_data(self, url , type="", d_st = False):
        json_df = pd.DataFrame(columns=self.col_df.index) 
        df_entry = pd.Series(index=self.col_df.index)

        df_entry = df_entry.fillna(0) 
        comp_out_count = Counter()
        
        if(isinstance(url, list)):json_data = url
        else:   
            json_str=open(url).read()
            json_data = json.loads(json_str)

        for i in range(len(json_data)):
            if i%500 == 0: print(i)
            df_entry *= 0
            m = str(json_data[i]["m"])
            df_entry.name = m
            for key in json_data[i]:
                if key == "m":  
                    pass            
                else: 
                    #key_wz = str(int(key)
                    key_wz = key
                    try:
                        ds_col = self.col_df.loc[key_wz]
                        #df_entry.loc[key_wz]
                        df_entry[key_wz] =  np.float32(json_data[i][key])
                    except: 
                        if d_st == True: 
                            print("column: {} not included in the input of: {}" .format(key_wz, m))
                        # comp_out_count[key_wz] +=1
            json_df = json_df.append(df_entry,ignore_index=False)
        # print("Counter of comp. not included :")
        # print(len(comp_out_count))
        # return json_df  
        return json_df.as_matrix().tolist()  


    def check_perf(self, lA, lB):
        assert(len(lA) == len(lB))
        gt3  = 0
        gtM = 0
        num = 0
        for i in range(len(lA)):
            num = abs(lA[i]-lB[i])
            if num > 3: gt3+=1
            if num > 10: gtM+=1
        return gt3, gtM
    

# DECLARATIONS

In [4]:
# Declarations
learning_rate = 0.001
n_classes   = 100    
n_input     = 1814 #1221
n_hidden_1  = 256   
n_hidden_2  = 256   
xtp1        = []  
ytp1        = []

l3 = 0; l15 = 0; start=0; elapsed_time=0;
rp_s = '';  tr_ac = ''; ev_ac = '';

def init():
    l3 = 0; l15 = 0; start=0; elapsed_time=0;
    rp_s = '';  tr_ac = ''; 

# col_df = pd.read_csv(COL_DS, index_col=0, sep=',', usecols=[0,1,2,3])
# print(col_df)

dataClass = fpDataModel( path= ALL_DS, norm = '', batch_size = 128, dType="classN", labelCol = 'FP_C', 
                         dataCol = 4,   nC=n_classes, nRange=1, toList = True )
print("data declared")

data declared


In [5]:
# get data from type = T, E 
print("start reading...")
start = time.time()
dataTrain,  dataEv =  dataClass.get_data(pathA=ALL_DS ) 
elapsed_time = float(time.time() - start)
print("data read - lenTrain={} - lenTests={} - time:{}" .format(len(dataTrain["label"]),len(dataEv["label"]),elapsed_time ))


start reading...
data read - lenTrain=4804 - lenTests=4806 - time:6.865105628967285


# <font color=red> JSON Reading is very SLOW!!! - only for go live not for training </font>
in order to create the new input format I need to read every single material and create the nes layout ... therefore it will take me 702seconds = 11min for 9.600 matnr ... <br> 
An alternative would be to create another program and generate the new excel in columns! 

In [6]:
# get data from json 

dataAll  = {'label' : [] , 'data' :  [] }
n_input2 = dataClass.set_columns(COL_DS)
print("input-no={}".format( n_input2))
json_str = '''[{ "m":"8989", "c1" :0.5 },
               { "m":"8988", "c3" :0.5 , "c4" :0.5 }] '''
json_data = json.loads(json_str)  #;print(json_data[0]['m'])
start = time.time()
dataAll['data'] = [555] #dataClass.feed_data(ALL_DSJ); print(dataAll['data'])
elapsed_time = float(time.time() - start)
print("data read - lenTrain={} - lenTests={} - time:{}" .format(len(dataTrain["label"]),len(dataEv["label"]),elapsed_time ))

# dataAll['label'] = [88,99]
    
# dataTest['data'] = dataAll['data']
# dataTest['data'] = dataAll['data']


input-no=1814
data read - lenTrain=4804 - lenTests=4806 - time:0.0


# NEXT DATA
the next that I should do regarding data handling is recording the type of data I am getting!<br>
Also - Functions to convert data into standard excel - matnr per column... 

# MODEL - CLASS!  <a id="model"/>

[go back](#index)

create my network class and don't initialize it ... only 1

In [49]:
class fpNN:
    def __init__(self, ncol, layers=2, hidden_nodes = [256 , 256], 
                 learning_rate = 0.1, min_count = 10, polarity_cutoff = 0.1       ):
        
        output = 100
        self.init_network(ncol, layers, hidden_nodes, output, learning_rate)
    
    def init_network(self, n_input, layers, hidden_nodes, n_classes, learning_rate):
        self.learning_rate = learning_rate # 0.001
        self.hidden_nodes = hidden_nodes
        self.n_input     = n_input #1814 #1221
        self.n_classes   = n_classes 

        # cust - network 
        self.x = tf.placeholder(tf.float32,   shape=[None, n_input],   name="x")
        self.y = tf.placeholder(tf.int16,     shape=[None, n_classes], name="cat")

        self.weights = {
            'h1': tf.Variable(tf.random_normal([n_input, hidden_nodes[0] ]),    name="Weights_1"),
            'h2': tf.Variable(tf.random_normal([ hidden_nodes[0], hidden_nodes[1]]), name="Weights_2"),
            'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name="Weights_out"),
        }
        self.biases = {
            'b1': tf.Variable(tf.random_normal([hidden_nodes[0]]), name="Bias_1"),
            'b2': tf.Variable(tf.random_normal([hidden_nodes[1]]), name="Bias_2"),
            'out': tf.Variable(tf.random_normal([n_classes]), name="Bias_out"),
        }
        
#         hidden_nodes = [1221,256,256,100]
#         self.weights = {} 
#         self.biases = {}
#         for (i in range(layers)):
#             self.weights["h"+string(i)] = tf.Variable(tf.random_normal([n_input, hidden_nodes[0] ]),name="Weights_1"
                
        # Hidden layer with RELU activation
        with tf.name_scope("fc_1"):
            self.layer_1 = tf.add(tf.matmul(self.x, self.weights['h1']), self.biases['b1'])
            self.layer_1 = tf.nn.relu(self.layer_1)
        # Hidden layer with RELU activation
        with tf.name_scope("fc_2"):
            self.layer_2 = tf.add(tf.matmul(self.layer_1, self.weights['h2']), self.biases['b2'])
            self.layer_2 = tf.nn.relu(self.layer_2)
        # Output layer with linear activation
        with tf.name_scope("fc_output"):
            self.pred = tf.matmul(self.layer_2, self.weights['out']) + self.biases['out']

        self.softmaxT = tf.nn.softmax(self.pred, )
        self.prediction=tf.reduce_max(self.y,1)

        with tf.name_scope("accuracy"):
            self.correct_prediction = tf.equal(tf.argmax(self.pred, 1), tf.argmax(self.y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
            tf.summary.scalar("accuracy", self.accuracy)

        with tf.name_scope("xent"):
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.pred, labels=self.y))
            tf.summary.scalar("xent", self.cost)

        with tf.name_scope("train"):
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
        
        self.summ = tf.summary.merge_all()
        self.init = tf.global_variables_initializer()
        self.saver= tf.train.Saver()
        
nc = fpModel(1, 2, 1814, learning_rate = 0.01)


In [61]:

class fpModel:
    def __init__(self, nn, model_path ):
        self.nn = nn
        self.model_path = model_path
    
    def get_nns(self):
        ns = "n:" + str(self.nn.n_input) + "*"
#       for(i in range(self.layers) )                
        ns = ns + str(self.nn.hidden_nodes[0]) + "*" + str(self.nn.hidden_nodes[1]) + "*"
        ns = ns + str(self.nn.n_classes)
        return ns 
    
    def dummy(self):
        print("hello")
        
    def restore_model(self, sess):
        print("Model restored from file: %s" % self.model_path)
        self.saver.restore(sess, self.model_path)
        
    def check_perf_CN(self, predv, dataEv ):
        pred_val = []
        data_val = []
        predvList = predv.tolist()
        print("denormalization all Evaluation : {} = {}" .format(len(predv), len(dataEv["label"])))
        for i in range(len(predv)):
        #for i in range(100):
            if (i % 1000==0): print(i , end="")
            pred_vali = 0; data_vali = 0;
            try:
                pred_vali = dataClass.deClassifN( predv.tolist()[i], np.max(predv[i]))
                data_vali = dataClass.deClassifN( dataEv['label'][i])
                # print("realVal: {} -- PP value: {}".format(data_vali,pred_vali))
                pred_val.append(pred_vali)
                data_val.append(data_vali)
            except:
                print("error: i={}, pred={}, data={} -- ".format(i, pred_vali, data_vali))

        l3, l15 = dataClass.check_perf(pred_val, data_val)  
        print("Total: {} GT3: {}  GTM: {}".format(len(pred_val), l3, l15)) 

    def train(self, dataTrain):
        training_iters = 10000 #200000
        batch_size = 128
        display_step = training_iters*0.1 #10%
        record_step  = training_iters*0.05 

        with tf.Session() as sess:
            sess.run(self.nn.init)
            start = time.time()
            
            for i in range(training_iters): 
                xtb, ytb = dataClass.next_batch(batch_size, dataTrain['data'], dataTrain['label']) 
                elapsed_time = float(time.time() - start)
                reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0

                if i % record_step == 0:
                    [train_accuracy] = sess.run([self.nn.accuracy], feed_dict={self.nn.x: xtb, self.nn.y: ytb }) 
        #             writer.add_summary(s, i)

                if i % display_step == 0:
                    print("step %d, training accracy %g " %(i, train_accuracy))

                sess.run(self.nn.optimizer, feed_dict={self.nn.x: xtb, self.nn.y: ytb})

            print("Optimization Finished!")
            save_path = self.nn.saver.save(sess, self.model_path)
            print("Model saved in file: %s" % save_path)
            print("Testing Accuracy:", sess.run(self.nn.accuracy, feed_dict={self.nn.x: dataEv['data'], 
                                                                             self.nn.y: dataEv['label']}))
    
    def evaluate(self, dataTrain, dataEv ):
        print("EVALUATION...")
        with tf.Session() as sess:
            sess.run(self.init)
            self.restore_model(sess)
            # test the model
            print("Training   Accuracy:", sess.run(self.nn.accuracy, feed_dict={self.x: dataTrain['data'], 
                                                                             self.y: dataTrain['label']}))
            print("Evaluation Accuracy:", sess.run(self.nn.accuracy, feed_dict={self.x: dataEv['data'],    
                                                                             self.y: dataEv['label']}))

            # xtp1.append(dataTest['data'][i]);    ytp1.append(dataTest['label'][i])
            predv, softv = sess.run([self.nn.pred, self.nn.softmaxT], feed_dict={x: dataEv['data']}) 

            print("Preview the first predictions:")
            for i in range(20):
                print("RealVal: {}  - PP value: {}".format( dataClass.deClassifN( dataEv['label'][i]), 
                                                           dataClass.deClassifN( predv.tolist()[i], np.max(predv[i]))  ))
                # maxa = sess.run([prediction], feed_dict={y: predv })

            self.check_perf_CN(predv, dataEv )
        
    def test(self, p_json_str=0, p_label=0):
        print("TESTS...")    
        dataTest = {'label' : [] , 'data' :  [] }
        pred_val = []
        print("input-no={}".format(dataClass.set_columns(COL_DS) ))
        
        if p_json_str != 0: json_str = p_json_str
        else: 
            json_str = '''[{ "m":"8989", "c1" :0.5 },
            { "m":"8988", "c3" :0.5 , "c4" :0.5 }] '''

        if p_json_str != 0: dataTest['label'] = p_label
        else: 
            dataTest['label'] = [88,99]
        
        json_data = json.loads(json_str)
        dataTest['data'] = dataClass.feed_data(json_data) 
        
        with tf.Session() as sess:
            sess.run(self.nn.init)
            self.restore_model(sess, model_path)
            predv = sess.run( self.nn.pred , feed_dict={x: dataTest['data']}) 

        for i in range(len(predv)):
            print("RealVal: {}  - PP value: {}".format( dataClass.deClassifN( dataEv['label'][i]), 
                                                       dataClass.deClassifN( predv.tolist()[i], np.max(predv[i]))  ))   

In [62]:
    def dummy2(self): # I CANT DECLARE METHODS FOR A CLASS IN DIFFERENT CELLS! 
        print("hello2")
        
model_path  = LOGDIR + "0F2CR3/model.ckpt"      
mlp =  fpModel(nc, model_path)
print(mlp.get_nns())
# mlp.dummy2()

n:1814*256*256*100


#### testing!!!!

In [ ]:
# mlp.model_path = LOGDIR + "0F2CR3/model.ckpt"  
mlp.train(dataTrain=dataTrain)

step 0, training accracy 0 


In [48]:
# evaluate 
mlp.evaluate(dataTrain, dataEv )

EVALUATION...
Model restored from file: ../../_zfp/data/my_graph/0F2CR3/model.ckpt
INFO:tensorflow:Restoring parameters from ../../_zfp/data/my_graph/0F2CR3/model.ckpt


NotFoundError: Key Weights_out_14/Adam not found in checkpoint
	 [[Node: save_13/RestoreV2_244 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_13/Const_0_0, save_13/RestoreV2_244/tensor_names, save_13/RestoreV2_244/shape_and_slices)]]

Caused by op 'save_13/RestoreV2_244', defined at:
  File "c:\users\extig\appdata\local\continuum\anaconda3\envs\udacity\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\extig\appdata\local\continuum\anaconda3\envs\udacity\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\extig\appdata\local\continuum\anaconda3\envs\udacity\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\extig\appdata\local\continuum\anaconda3\envs\udacity\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\extig\appdata\local\continuum\anaconda3\envs\u

In [44]:
mlp.test()

TESTS...
input-no=1814
0
Model restored from file: ../../_zfp/data/my_graph/0F2CR3/model.ckpt
INFO:tensorflow:Restoring parameters from ../../_zfp/data/my_graph/0F2CR3/model.ckpt


NameError: name 'pred' is not defined

# Other <a id="o"/>

[go back](#index)



In [7]:
print("NETWORK") 
x = tf.placeholder(tf.float32,   shape=[None, n_input],   name="x")
y = tf.placeholder(tf.int16,     shape=[None, n_classes], name="cat")

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]),    name="Weights_1"),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name="Weights_2"),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name="Weights_out"),
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name="Bias_1"),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name="Bias_2"),
    'out': tf.Variable(tf.random_normal([n_classes]), name="Bias_out"),
}

# Hidden layer with RELU activation
with tf.name_scope("fc_1"):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
# Hidden layer with RELU activation
with tf.name_scope("fc_2"):
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
# Output layer with linear activation
with tf.name_scope("fc_output"):
    pred = tf.matmul(layer_2, weights['out']) + biases['out']

# - declaration of model and global attributes 
# pred = multilayer_perceptron(x, weights, biases)

softmaxT = tf.nn.softmax(pred, )
prediction=tf.reduce_max(y,1)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)

with tf.name_scope("xent"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    tf.summary.scalar("xent", cost)
    
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
summ = tf.summary.merge_all()
init = tf.global_variables_initializer()
saver = tf.train.Saver()
print("done")

NETWORK
done


In [19]:

def make_hparam_string(learning_rate, no_fc):
    return "lr_%.0E,fc=%d" % (learning_rate, no_fc) 

hparam = make_hparam_string(learning_rate, 3)

## Record logs automatically

In [30]:
LOG        = "../../_zfp/LOGT.txt"

def recordLogF(date ='', time='', it=1000, nn='', lr=0.01, typ='TR', DS='', AC=0, num=0, AC3=0, AC10=0, desc=''):
    
    if desc == '': print("Log not recorded"); return 
    
    f= open(LOG ,"a+") #w,a,
    if date != '':
        dats = date
    else: dats = datetime.now().strftime('%d.%m.%Y') 
   
    if time != '':
        times = time
    else: times = datetime.now().strftime('%H:%M:%S') 
    
    
    line = dats + '\t' + times
    line = line + '\t' + str(it) + '\t'+ nn +  '\t' + str(lr)
    line = line + '\t' + typ 
    line = line + '\t' + str(DS) + '\t' + str(AC) + '\t' + str(num) + '\t' + str(AC3) + '\t' +  str(AC10) + '\t' + desc + '\n'
    
    f.write(line)
    f.close()
    print("___Log recorded")

# recordLogF( it=1000, nn='100*100', lr=0.01, typ='TR', DS=DSC, AC=0.99, num=400, AC3=4, AC10=3, desc='test f1')
# recordLogF( it=1000, nn='100*100', lr=0.01, typ='TR', DS=DSC, AC=0.99, num=400, AC3=4, AC10=3, desc='test f2')
# recordLogF( it=1000, nn='100*100', lr=0.01, typ='TR', DS=DSC, AC=0.99, num=400, AC3=4, AC10=3, desc='test f3')


In [25]:
def check_perf_CN(predv, dataEv ):
    pred_val = []
    data_val = []
    predvList = predv.tolist()
    print("denormalization all Evaluation : {} = {}" .format(len(predv), len(dataEv["label"])))
    for i in range(len(predv)):
    #for i in range(100):
        if (i % 1000==0): print(i , end="")
        pred_vali = 0; data_vali = 0;
        try:
            pred_vali = dataClass.deClassifN( predv.tolist()[i], np.max(predv[i]))
            data_vali = dataClass.deClassifN( dataEv['label'][i])
            # print("realVal: {} -- PP value: {}".format(data_vali,pred_vali))
            pred_val.append(pred_vali)
            data_val.append(data_vali)
        except:
            print("error: i={}, pred={}, data={} -- ".format(i, pred_vali, data_vali))

    l3, l15 = dataClass.check_perf(pred_val, data_val)  
    print("Total: {} GT3: {}  GTM: {}".format(len(pred_val), l3, l15)) 

# TR <a id="tr"/>

[go back](#index)

In [20]:
# train
print("TRAINING...")
model_path  = LOGDIR + "0F2CR2/model.ckpt"
training_iters = 10000 #200000

def train_model(hparam): 
    batch_size = 128
    display_step = training_iters*0.1 #10%
    record_step  = training_iters*0.05 
    return
    with tf.Session() as sess:
        # Initialize variables
        sess.run(init)
#         writer = tf.summary.FileWriter(LOGDIR + hparam)
#         writer.add_graph(sess.graph)
        start = time.time()
        for i in range(training_iters): 

            xtb, ytb = dataClass.next_batch(batch_size, dataTrain['data'], dataTrain['label']) 
            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            if i % record_step == 0:
                [train_accuracy] = sess.run([accuracy], feed_dict={x: xtb, y: ytb }) 
    #             writer.add_summary(s, i)

            if i % display_step == 0:
#                 print("step %d, training accracy %g " %(i, train_accuracy))    
                elapsed_time = float(time.time() - start)
                reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
                rp_s = str(reviews_per_second)[0:5]
                tr_ac = str(train_accuracy)[:5]  

                print('step {} - %Speed(it/disp_step): {} - tr_ac {}' .format(i, rp_s, tr_ac ))
                      
            sess.run(optimizer, feed_dict={x: xtb, y: ytb})

        print("Optimization Finished!")
        save_path = saver.save(sess, model_path)
        print("Model saved in file: %s" % save_path)
                
train_model(hparam)

TRAINING...


In [31]:
recordLogF( it=1000, nn=mlp.get_nns(), lr=mlp.learning_rate, typ='TR', DS=DSC, AC=tr_ac, 
           num=len(dataTrain["label"]), AC3=0, AC10=0, desc='C100 - All')


# ev_ac = str(  sess.run(accuracy, feed_dict={x: dataEv['data'], y: dataEv['label']})  )[:5]  

print("Ev Accuracy:", ev_ac )

recordLogF( it=1000, nn=mlp.get_nns(), lr=mlp.learning_rate, typ='EV', DS=DSC, AC=tr_ac, 
           num=len(dataEv["label"]), AC3=0, AC10=0, desc='C100 - All')

___Log recorded
Ev Accuracy: 
___Log recorded


# EV <a id="ev"/>

[go back](#index)

In [18]:
def restore_model(sess,model_path):
#   print(model_path)
    saver.restore(sess, model_path)
#     saver.restore(sess, "../../_zfp/data/my_graph/0F2CR2/model.ckpt")

In [17]:
def evaluate_model():
    print("EVALUATION...")
    model_path  = LOGDIR + "0F2CR2/model.ckpt"

    return
    with tf.Session() as sess:
        print("Model restored from file: %s" % model_path)

        sess.run(init)
        restore_model(sess, model_path)
        

        # test the model
        print("Training   Accuracy:", sess.run(accuracy, feed_dict={x: dataTrain['data'], y: dataTrain['label']}))
        print("Evaluation Accuracy:", sess.run(accuracy, feed_dict={x: dataEv['data'],    y: dataEv['label']}))

        # xtp1.append(dataTest['data'][i]);    ytp1.append(dataTest['label'][i])
        predv, softv = sess.run([pred, softmaxT], feed_dict={x: dataEv['data']}) 
        
        print("Preview the first predictions:")
        for i in range(20):
            print("RealVal: {}  - PP value: {}".format( dataClass.deClassifN( dataEv['label'][i]), 
                                                       dataClass.deClassifN( predv.tolist()[i], np.max(predv[i]))  ))
            # maxa = sess.run([prediction], feed_dict={y: predv })
        
        check_perf_CN(predv, dataEv )
        
#         pred_val = []
#         data_val = []
#         predvList = predv.tolist()
#         print("denormalization all Evaluation : {} = {}" .format(len(predv), len(dataEv["label"])))
#         for i in range(len(predv)):
#         #for i in range(100):
#             if (i % 1000==0): print(i)
#             pred_vali = 0; data_vali = 0;
#             try:
#                 pred_vali = dataClass.deClassifN( predv.tolist()[i], np.max(predv[i]))
#                 data_vali = dataClass.deClassifN( dataEv['label'][i])
#                 # print("realVal: {} -- PP value: {}".format(data_vali,pred_vali))
#                 pred_val.append(pred_vali)
#                 data_val.append(data_vali)
#             except:
#                 print("error: i={}, pred={}, data={} -- ".format(i, pred_vali, data_vali))
        
#         l3, l15 = dataClass.check_perf(pred_val, data_val)  
#         print("Total: {} GT3: {}  GTM: {}".format(len(pred_val), l3, l15))     

evaluate_model()



EVALUATION...


# TESTS! <a id="ts"/>

[go back](#index)

In [15]:
def test_model():
    print("TESTS...")    
    dataTest = {'label' : [] , 'data' :  [] }
    pred_val = []

    print("input-no={}".format(dataClass.set_columns(COL_DS) ))
        
    json_str = '''[{ "m":"8989", "c1" :0.5 },
    { "m":"8988", "c3" :0.5 , "c4" :0.5 }] '''
    
    json_data = json.loads(json_str)
    dataTest['data'] = dataClass.feed_data(json_data) 
    dataTest['label'] = [88,99]
    
    with tf.Session() as sess:
        print("Model restored from file: %s" % model_path)

        sess.run(init)
        restore_model(sess)
        predv = sess.run( pred , feed_dict={x: dataTest['data']}) 
    
    for i in range(len(predv)):
        print("RealVal: {}  - PP value: {}".format( dataClass.deClassifN( dataEv['label'][i]), 
                                                   dataClass.deClassifN( predv.tolist()[i], np.max(predv[i]))  ))    
    
test_model()



TESTS...
input-no=1814
Model restored from file: ../../_zfp/data/my_graph/0F2CR2/model.ckpt
../../_zfp/data/my_graph/0F2CR2/model.ckpt
INFO:tensorflow:Restoring parameters from ../../_zfp/data/my_graph/0F2CR2/model.ckpt
RealVal: 63  - PP value: 93
RealVal: 93  - PP value: 99


In [ ]:
def test_model2():
    print("TESTS...")    
    dataTest = {'label' : [] , 'data' :  [] }
    pred_val = []
    
    # COMPRESS INTO GET_DATA2
    print("input-no={}".format(dataClass.set_columns(COL_DS) ))
    dataTest['data']  = dataClass.feed_data(ALL_DSJ) 
    dataTest['label'] = [88,99]  # READ FROM FILE! 
    
    with tf.Session() as sess:
        print("Model restored from file: %s" % model_path)
        sess.run(init)
        restore_model(sess)
        predv = sess.run( pred , feed_dict={x: dataTest['data']}) 
    
    for i in range(len(predv)):
        print("RealVal: {}  - PP value: {}".format( dataClass.deClassifN( dataEv['label'][i]), 
                                                   dataClass.deClassifN( predv.tolist()[i], np.max(predv[i]))  ))    
    
test_model2()

